# Part-Of-Speech (POS) Tagging

In [1]:
import nltk
from sklearn.model_selection import train_test_split

# Download the dataset
nltk.download('treebank')
nltk.download('universal_tagset')

# Load the dataset
dataset = nltk.corpus.treebank.tagged_sents(tagset='universal')

# Split dataset into training and test sets
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

print("Training set size:", len(train_data))
print("Test set size:", len(test_data))
print("Sample data:\n", train_data[0])

[nltk_data] Downloading package treebank to /home/omar/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /home/omar/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


Training set size: 3131
Test set size: 783
Sample data:
 [('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')]


## 1. Hidden Markov Model (HMM) Overview 📚

Hidden Markov Model (HMM) is a statistical model that is used to describe the probability distribution of observable data. It is a generative model that assumes that the observed data is generated by a sequence of hidden states. The hidden states are not directly observable, but the observable data is generated based on the hidden states.

## 2. Building the HMM 🛠️

In [2]:
from collections import defaultdict

# Initialize the HMM parameters
transition_probs = defaultdict(lambda: defaultdict(lambda: 0))
emission_probs = defaultdict(lambda: defaultdict(lambda: 0))
initial_probs = defaultdict(lambda: 0)
tag_count = defaultdict(lambda: 0)
vocab = set()

# Calculate initial, transition, and emission probabilities
for sentence in train_data:
    prev_tag = None
    for word, tag in sentence:
        vocab.add(word)
        tag_count[tag] += 1
        emission_probs[tag][word] += 1
        if prev_tag is None:
            initial_probs[tag] += 1
        else:
            transition_probs[prev_tag][tag] += 1
        prev_tag = tag

# Normalize the probabilities
for tag in initial_probs:
    initial_probs[tag] /= len(train_data)

for prev_tag in transition_probs:
    total_transitions = sum(transition_probs[prev_tag].values())
    for tag in transition_probs[prev_tag]:
        transition_probs[prev_tag][tag] /= total_transitions

for tag in emission_probs:
    total_emissions = sum(emission_probs[tag].values())
    for word in emission_probs[tag]:
        emission_probs[tag][word] /= total_emissions

print("Initial probabilities:\n", dict(initial_probs))
print("Sample transition probabilities:\n", dict(transition_probs['NOUN']))
print("Sample emission probabilities:\n", dict(emission_probs['NOUN']))

Initial probabilities:
 {'NOUN': 0.29255828808687323, 'CONJ': 0.051421271159374005, 'DET': 0.2293197061641648, '.': 0.08176301501117854, 'ADP': 0.12583839029064198, 'ADJ': 0.04247844139252635, 'ADV': 0.05429575215586075, 'X': 0.024912168636218462, 'PRON': 0.07601405301820505, 'NUM': 0.010220376876397317, 'VERB': 0.009581603321622485, 'PRT': 0.0015969338869370809}
Sample transition probabilities:
 {'NOUN': 0.2627788205508752, '.': 0.2419136583875333, 'ADJ': 0.011698459120869528, 'ADP': 0.1760006984154699, 'NUM': 0.0097341656117683, 'VERB': 0.14740931511632982, 'CONJ': 0.04225413592911083, 'PRT': 0.04430573137194989, 'ADV': 0.016718320310794885, 'X': 0.029202496835304903, 'DET': 0.013182591994412676, 'PRON': 0.004801606355580776}
Sample emission probabilities:
 {'Pierre': 4.362240446693422e-05, 'Vinken': 8.724480893386844e-05, 'years': 0.004274995637759554, 'board': 0.0011341825161402897, 'director': 0.001221427325074158, 'Nov.': 0.001003315302739487, 'dealers': 0.0004798464491362764, 'd

## 3. Implementing the Viterbi Algorithm 🔄

In [3]:
def viterbi_algorithm(sentence, initial_probs, transition_probs, emission_probs, tag_count):
    states = list(tag_count.keys())
    V = [{}]
    path = {}

    # Initialize base cases (t == 0)
    for state in states:
        V[0][state] = initial_probs[state] * emission_probs[state].get(sentence[0], 1e-6)
        path[state] = [state]

    # Run Viterbi for t > 0
    for t in range(1, len(sentence)):
        V.append({})
        new_path = {}

        for state in states:
            (prob, prev_state) = max(
                (V[t-1][prev_state] * transition_probs[prev_state].get(state, 1e-6) * 
                 emission_probs[state].get(sentence[t], 1e-6), prev_state)
                for prev_state in states
            )
            V[t][state] = prob
            new_path[state] = path[prev_state] + [state]

        path = new_path

    # Find the best path
    (prob, best_state) = max((V[-1][state], state) for state in states)
    return path[best_state], prob

# Test the Viterbi algorithm on a sample sentence
sample_sentence = [word for word, _ in test_data[0]]
predicted_tags, probability = viterbi_algorithm(sample_sentence, initial_probs, transition_probs, emission_probs, tag_count)

print("Sample Sentence:", sample_sentence)
print("Predicted Tags:", predicted_tags)
print("Probability of the sequence:", probability)

Sample Sentence: ['For', 'the', 'Agency', 'for', 'International', 'Development', ',', 'appropriators', 'approved', '$', '200', 'million', '*U*', 'in', 'secondary', 'loan', 'guarantees', 'under', 'an', 'expanded', 'trade', 'credit', 'insurance', 'program', ',', 'and', 'total', 'loan', 'guarantees', 'for', 'the', 'Overseas', 'Private', 'Investment', 'Corp.', 'are', 'increased', '*-3', 'by', '$', '40', 'million', '*U*', 'over', 'fiscal', '1989', 'as', 'part', 'of', 'the', 'same', 'Poland', 'package', '.']
Predicted Tags: ['ADP', 'DET', 'NOUN', 'ADP', 'NOUN', 'NOUN', '.', 'PRON', 'VERB', '.', 'NUM', 'NUM', 'X', 'ADP', 'ADJ', 'NOUN', 'NOUN', 'ADP', 'DET', 'VERB', 'NOUN', 'NOUN', 'NOUN', 'NOUN', '.', 'CONJ', 'ADJ', 'NOUN', 'NOUN', 'ADP', 'DET', 'ADJ', 'ADJ', 'NOUN', 'NOUN', 'VERB', 'VERB', 'X', 'ADP', '.', 'NUM', 'NUM', 'X', 'ADP', 'ADJ', 'NUM', 'ADP', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'NOUN', '.']
Probability of the sequence: 2.498937212411083e-162


## 4. Model Evaluation 📊

In [4]:
correct = 0
total = 0

for sentence in test_data:
    words = [word for word, tag in sentence]
    true_tags = [tag for word, tag in sentence]
    predicted_tags, _ = viterbi_algorithm(words, initial_probs, transition_probs, emission_probs, tag_count)
    
    correct += sum(p == t for p, t in zip(predicted_tags, true_tags))
    total += len(sentence)

accuracy = correct / total
print("POS Tagging Accuracy:", accuracy)

POS Tagging Accuracy: 0.935081999124045


## Conclusion
In this notebook, we built a Part-of-Speech tagger using Hidden Markov Models (HMMs) and the Viterbi algorithm. The model was trained on the Treebank dataset and successfully tagged parts of speech with reasonable accuracy. This approach provides a fundamental understanding of sequence models and their application in NLP tasks.